In [1]:
import copy
import sys
import os

In [2]:
class Init(type):
    def __call__(cls, color, x, y, isAlive = True):
        instance = cls.__new__(cls)
        cls.instances.append(instance)

        instance.color = color
        instance.x = x
        instance.y = y
        instance.isAlive = isAlive
        instance.move_counter = 0
        if instance.black_motions == []:
            instance.black_motions = instance.white_motions

        return instance

In [3]:
class Figure(object):
    white_motions = []
    black_motions = []
    can_jump_over_other_figures = False

    def __init__(*args):
        pass

    def getName(self):
        return self.name.upper() if self.color else self.name

    def getPossibleMotions(self):
        # motions = [[x, y], ...]
        motions = self.white_motions if self.color else self.black_motions
        possible_motions = []
        # отсекли все ходы, не попавшие в поле
        for motion in [[self.x + motion[0], self.y + motion[1]] for motion in motions]:
            if -1 < motion[0] < 8 and -1 < motion[1] < 8:
                possible_motions.append(motion)
      
        return possible_motions

    def getMotions(self):
        """ Выдает координаты возможных ходов"""
        global game_history
        global current_motion

        available_motions = []
        for motion in self.getPossibleMotions():
            if self.can_jump_over_other_figures:
                figure = game_history[current_motion][motion[1]][motion[0]]
            #if (figure == [] and motion[3]) or (figure != [] and figure.color == (1 - self.color) and motion[2]):
            if figure == [] or not figure.isAlive or (figure.color == (1 - self.color)):
                available_motions.append(motion)
        # если фигура не может прыгать через другие, значит на ее пути не должно быть других фигур
        else:
            # генерируем клетки, через которуе нужно данной фигуре пройти, чтобы попасть в указанную позицию
            # приращение по координатам: self.x - motion[0], self.y - motion[1]
           # т.к. все фигуры, кроме коня, движутся линейно
            delta_x = motion[0] - self.x
            delta_y = motion[1] - self.y
            motion_cells = []

            if delta_x > 0:
                range_x = range(1, delta_x + 1)
            elif delta_x < 0:
                range_x = reversed(range(delta_x, 0))
            else:
                range_x = [0] * abs(delta_y)

            if delta_y > 0:
                range_y = range(1, delta_y + 1)
            elif delta_y < 0:
                range_y = reversed(range(delta_y, 0))
            else:
                range_y = [0] * abs(delta_x)

            motion_cells = list(zip(range_x, range_y))
            # проверяем, чтобы на пути не встретился противник
            # если противник встретился на промежутной, т.е. не последней, значит этот ход не доступен
            flag = True
            # print(self.color, self.name, self.x, self.y, motion, motion_cells)
            for cell in motion_cells:
                figure = game_history[current_motion][self.y + cell[1]][self.x + cell[0]]
                if (flag and figure != [] and figure.isAlive and cell != motion_cells[-1]):
                    flag = False
            if flag and (figure == [] or figure.color != self.color):
                available_motions.append(motion)

        return available_motions

    def isCorrectMotion(self, x, y):
        for motions in self.getMotions():
            if motions[0] == x and motions[1] == y:
                return True
        return False

    def move(self, x, y):
        global game_history
        global current_motion

        if game_history[current_motion][y][x] != []:
            game_history[current_motion][y][x].isAlive = False

        game_history[current_motion][y][x], game_history[current_motion][self.y][self.x] = self, []

        self.x = x
        self.y = y
        self.move_counter += 1      

In [4]:
class King(Figure, metaclass = Init):
    """ Король """
    instances = []
    name = 'k'
    rus_name = 'король'
    white_motions = [[1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0]]

In [5]:
class Bishop(Figure, metaclass = Init):
    """ Слон """
    instances = []
    name = 'b'
    rus_name = 'слон'
    white_motions = [[1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7],
                     [-1, 1], [-2, 2], [-3, 3], [-4, 4], [-5, 5], [-6, 6], [-7, 7],
                     [-1, -1], [-2, -2], [-3, -3], [-4, -4], [-5, -5], [-6, -6], [-7, -7],
                     [1, -1], [2, -2], [3, -3], [4, -4], [5, -5], [6, -6], [7, -7]]

    # слон не может пригать через другие фигуры 

In [6]:
class Rook(Figure, metaclass = Init):
    """ Ладья """
    instances = []
    name = 'r'
    rus_name = 'ладья'
    white_motions = [[1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0],
                     [-1, 0], [-2, 0], [-3, 0], [-4, 0], [-5, 0], [-6, 0], [-7, 0],
                     [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7],
                     [0, -1], [0, -2], [0, -3], [0, -4], [0, -5], [0, -6], [0, -7]]

In [7]:
class Queen(Figure, metaclass = Init):
    """ Ферзь """
    instances = []
    name = 'q'
    rus_name = 'ферзь'
    white_motions = Bishop.white_motions + Rook.white_motions

In [8]:
class Knight(Figure, metaclass = Init):
    """ Конь """
    instances = []
    name = 'n'
    rus_name = 'конь'
    can_jump_over_other_figures = True
    white_motions = [[1, 2], [-1, 2], [1, -2], [-1, -2],
                      [2, 1], [-2, 1], [2, -1], [-2, -1]]

In [9]:
class Pawn(Figure, metaclass = Init):
    """ Пешка """
    instances = []
    name = 'p'
    rus_name = 'пешка'
    white_motions = [[0, 1], [0, 2], [1, 1], [-1, 1]]
    black_motions = [[0, -1], [0, -2], [1, -1], [-1, -1]]

    def getMotions(self):
        global game_history
        global current_motion

        possible_motions = []
        for motion in super().getMotions():
            figure = game_history[current_motion][motion[1]][motion[0]]
            if self.move_counter > 0 and abs(motion[1] - self.y) == 2:
                pass
            elif abs(motion[0] - self.x) == 1 and abs(motion[1] - self.y) == 1 and figure == []:
                pass
            elif abs(motion[0] - self.x) == 0 and (abs(motion[1] - self.y) == 1 or abs(motion[1] - self.y) == 2) and figure != []:
                pass
            else:
                possible_motions.append(motion)
            return possible_motions

In [10]:
# инициализируем шахматное поле
def init_field():
    global game_history
    # заполняем стартовую позицию
    field = [[[] for __ in range(8)] for _ in range(8)]
    for i in range(len(field)):
        # белые
        if i == 0:
            # ладья
            field[i][0] = Rook(1, 0, i)
            # ладья
            field[i][7] = Rook(1, 7, i)
            # конь
            field[i][1] = Knight(1, 1, i)
            # конь
            field[i][6] = Knight(1, 6, i)
            # слон
            field[i][2] = Bishop(1, 2, i)
            # слон
            field[i][5] = Bishop(1, 5, i)
            # ферзь
            field[i][3] = Queen(1, 3, i)
            # король
            field[i][4] = King(1, 4, i)
        # черные
        if i == 7:
            # ладья
            field[i][0] = Rook(0, 0, i)
            # ладья
            field[i][7] = Rook(0, 7, i)
            # конь
            field[i][1] = Knight(0, 1, i)
            # конь
            field[i][6] = Knight(0, 6, i)
            # слон
            field[i][2] = Bishop(0, 2, i)
            # слон
            field[i][5] = Bishop(0, 5, i)
            # ферзь
            field[i][3] = Queen(0, 3, i)
            # король
            field[i][4] = King(0, 4, i)
        # пешки белые
        if i == 1:
            for j in range(8):
                field[i][j] = Pawn(1, j, i)
        # пешки черные
        if i == 6:
            for j in range(8):
                field[i][j] = Pawn(0, j, i)
              
    game_history.append(field)
    #game_history[0]

In [11]:
class_assoc = {'P': Pawn, 'B': Bishop, 'N': Knight, 'R': Rook, 'Q': Queen, 'K': King}

In [14]:
# инициалищируем партию
# для каждого хода получаем все экземпляры нужного класса, 
# проверяем, кто мог сходить таким образом и ходим
def init_notation():
    global game_history, motions, motions_not_parsed, current_motion
    #print(motions)
    for motion in motions:
        for color in [1, 0]:
            game_history.append(copy.deepcopy(game_history[current_motion]))
            current_motion += 1
            #print(color)
            #print(motion)
            if color == 0 and len(motion) == 1:
                continue
            if type(motion[not color]) != list:
                # если рокировка
                if motion[not color].upper() == 'OO':
                # если у король еще не ходил
                    for king in King.instances:
                        if king.color == color and king.move_counter == 0:
                        # если есть ладья, которая еще не ходила
                            for rook in Rook.instances:
                                if rook.color == color and rook.move_counter == 0:
                                # если это правая ладья, значит рокировка короткая
                                    if rook.x == 7:
                                    # если между ними есть 2 свободные клетки
                                        if game_history[current_motion][rook.y][5] == [] and \
                                            game_history[current_motion][rook.y][6] == []:
                                            # тогда делаем рокировку
                                            king.move(6, king.y)
                                            rook.move(5, king.y)
                                            break

                elif motion[not color].upper() == 'OOО':
                  # если у король еще не ходил
                    for king in King.instances:
                        if king.color == color and king.move_counter == 0:
                        # если есть ладья, которая еще не ходила
                            for rook in Rook.instances:
                                if rook.color == color and rook.move_counter == 0:
                                  # если это левая ладья, значит рокировка длинная
                                    if rook.x == 9:
                                    # если между ними есть 3 свободные клетки
                                        if game_history[current_motion][rook.y][1] == [] and \
                                            game_history[current_motion][rook.y][2] == [] and \
                                            game_history[current_motion][rook.y][3] == []:
                                            # тогда делаем рокировку
                                            king.move(2, king.y)
                                            rook.move(3, king.y)
                                            break
                    else:
                        break

    #print(motion)                  
    #print(color)

        if type(motion[not color]) != list:
            break
      
      #print(motion[color], class_assoc.get(motion[color][0]))

      #if 'a' <= motion[not color][0].lower() <= 'h':
      #  motion[not color][0] = 'P'

      #print(motion)
      #print(motion[not color])
      #print(class_assoc.get(motion[not color][0]))
      #print(class_assoc.get(motion[not color][0]).instances)

        for figure in class_assoc.get(motion[not color][0]).instances:
            if figure.color == color and figure.isCorrectMotion(motion[not color][1], motion[not color][2]):
                #print(figure.color, figure.x, figure.y, motion[not color][1], motion[not color][2], figure.getMotions())
                figure.move(motion[not color][1], motion[not color][2])
                break

    current_motion = 0

In [15]:
#color = 1
#for king in King.instances:
#    if king.color == color and king.move_counter == 0:
#        for rook in Rook.instances:
#            if rook.color == color and rook.move_counter == 0:
#                if rook.x == 0:
#                    if game_history[current_motion][rook.y][1] == [] and \
#                        game_history[current_motion][rook.y][2] == [] and \
#                        game_history[current_motion][rook.y][3] == []:
#                        king.move(2, king.y)
#                        rook.move(3, rook.y)
#                        print_field(game_history[current_motion])
#                        print(current_motion, game_history[current_motion][rook.y][3])

In [18]:
letters = '  A   B   C   D   E   F   G   H '
def print_field(step):
    global selected_cell, motions_not_parsed, game_history, available_motions, own_game
    if step > 0 and not own_game:
        print(motions_not_parsed[step - 1])
    else:
        print('Ход # ', step)
    x = [['.' for __ in range(8)] for _ in range(8)]
    for _y, row in enumerate(game_history[step]):
        x[_y] = ''
        for _x, cell in enumerate(row):
            if cell == []:
                if available_motions != [] and [_x, _y] in available_motions:
                    x[_y] += ' {.}'
                else:
                    x[_y] += '  . '
            else:
                if selected_cell != [] and selected_cell[0] == _x and selected_cell[1] == _y:
                    x[_y] += ' [' + cell.getName() + ']'
                else:
                    if available_motions != [] and [_x, _y] in available_motions:
                        x[_y] += ' {' + cell.getName() + '}'
                    else:
                        x[_y] += '  ' + cell.getName() + ' '

          # каждой фигуре на доске задаем ее координаты
            cell.x = _x
            cell.y = _y
            #x[y] = ' '.join(['.' if cell == [] else cell.getName() for cell in row])
    print('\n ', letters, '\n')
    print(8, x[7], ' 8', '\n')
    print(7, x[6], ' 7', '\n')
    print(6, x[5], ' 6', '\n')
    print(5, x[4], ' 5', '\n')
    print(4, x[3], ' 4', '\n')
    print(3, x[2], ' 3', '\n')
    print(2, x[1], ' 2', '\n')
    print(1, x[0], ' 1', '\n')
    print(' ', letters, '\n')
    #print('')

In [19]:
letters_to_number = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}

In [20]:
number_to_letter = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

In [21]:
# парсим файл с партией в короткой нотации
def load_short_notation(filename):
    global motions, motions_not_parsed, letters_to_number
    motions = []
    with open(filename) as file:
        motions_not_parsed = list(map(lambda x: x.strip(), file.read().split('\n')))
        for row in motions_not_parsed:
            motions.append(row)
          
    motions_not_parsed = []
    for i in range(len(motions)):
        motions[i] = motions[i].split(' ')[1:]
        motions_not_parsed.append(motions[i][0])
        motions_not_parsed.append(motions[i][1])
        #print(motions[i])
        #motions[i].pop(0)
        if len(motions[i]) > 1:
            motions[i][1] = motions[i][1]
        for word in range(len(motions[i])):
            for character in motions[i][word]:
                if character == 'x':
                    motions[i][word] = motions[i][word].replace(character, '')
                if character.isalnum() == False:
                    motions[i][word] = motions[i][word].replace(character, '')
                  
    for i in range(len(motions)):
        for j in range(len(motions[i])):
            if len(motions[i][j]) == 2 and motions[i][j] != 'OO' and \
                motions[i][j] != '10' and motions[i][j] != '01':
                motions[i][j] = 'P' + motions[i][j]
            if len(motions[i][j]) == 3:
                motions[i][j] = list(motions[i][j])
                motions[i][j][1] = letters_to_number.get(motions[i][j][1])
                motions[i][j][2] = int(motions[i][j][2]) - 1
            if motions[i][j][0] in number_to_letter:
                motions[i][j][0] = 'P'
  #motions_not_parsed = list(map(lambda x: x.split('-'), motions_not_parsed))